In [1]:
import random
from PIL import Image
import csv
import os
from pathlib import Path
import shutil


dirPath = Path("R:/shugatenFg/")
level_list = [0, 2 , 3]

class Element(object):
    def __init__(self):
        self.name = name
        self.xy = [(0, 0) for i in range(4)]


class Stand(object):
    def __init__(self, name):
        self.name = name
        self.size = [(0, 0) for _ in range(4)]
        self.eyebrow = {}
        self.eye = {}
        self.mouth = {}
        self.cheek = {}
        self.cloth = {}

    def set_size(self, data, level):
        self.size[level] = (int(data["width"]), int(d["height"]))


def check_type(word, data):
    return word in data["name"] and word != data["name"] and "なし" not in data["name"]


def get_data(data):
    return (data["layer_id"], int(data["top"]), int(data["left"]))


def add_data(container, data, level):
    id, x, y = [int(i) for i in get_data(data)]
    container.setdefault(id, [[id, 0, 0] for _ in range(4)])
    container[id][level] = [id, x, y]


stand_list = {}

for folder in dirPath.iterdir():
    for file in folder.iterdir():
        for level in level_list:
            if f"{level}.txt" in file.name:
                name = file.name.split("_")[0]
                stand = stand_list.setdefault(name, Stand(name))
                with open(file, "r", encoding="utf-16le") as f:
                    data = list(csv.DictReader(f, delimiter="\t"))
                    for d in data:
                        if d["name"] == "":
                            stand.set_size(d, level)
                        elif check_type("眉", d):
                            add_data(stand.eyebrow, d, level)
                        elif check_type("目", d):
                            add_data(stand.eye, d, level)
                        elif check_type("口", d):
                            add_data(stand.mouth, d, level)
                        elif check_type("頬", d):
                            add_data(stand.cheek, d, level)
                        elif any(
                            check_type(check, d) for check in ["正面", "斜め", "後ろ"]
                        ):
                            add_data(stand.cloth, d, level)

In [2]:
for level in level_list:
    folder = dirPath / str(level)
    if os.path.exists(folder):
        shutil.rmtree(folder)
    os.makedirs(folder, exist_ok=True)

for stand in stand_list.values():
    folder = stand.name[:-1]
    for cloth in stand.cloth.values():
        color = tuple([random.randrange(255) for _ in range(3)])
        elements = [
            random.choice(list(e.values()))
            for e in [stand.cheek, stand.mouth, stand.eye, stand.eyebrow]
            if len(e) > 0
        ]
        for level in level_list:
            base_img = Image.new(mode="RGB", size=stand.size[level], color=color)
            for e in [cloth, *elements]:
                img = Image.open(
                    dirPath / folder / f"{stand.name}_{level}_{e[level][0]}.png"
                )
                base_img.paste(img, (e[level][2], e[level][1]), img)

            base_img.save(dirPath / str(level) / f"{stand.name}_{cloth[level][0]}.png")
